In [ ]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# Función para obtener los FLOPs
def get_flops(model, input_shape):
    # Crear una entrada dummy con el tamaño del lote especificado
    input_data = tf.random.normal(input_shape)

    # Convertir el modelo en una función concreta
    concrete_function = tf.function(lambda x: model(x))
    concrete_function = concrete_function.get_concrete_function(input_data)

    # Convertir las variables del modelo en constantes
    frozen_func = convert_variables_to_constants_v2(concrete_function)

    # Utilizar el perfilador de TensorFlow para obtener los FLOPs
    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph, run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops

In [ ]:
def HaussDorff_Distance(model, X_test, Y_test, show = False):
    th = 0.3
    hausdorff_distance_sum = 0
    print(len(X_test))
    Y_PRED = []
    for img_test, img_label in zip(X_test, Y_test):
        #print(img_test)
        img_test = np.expand_dims(img_test, axis=0)
        img_predicted = (model.predict(img_test)>= th).astype(np.uint8)
        #Y_PRED.append(img_predicted)
        img_predicted = img_predicted[0,:, :,0]
        if show == True:
            plt.subplot(1, 2, 1), plt.imshow(img_predicted, cmap='gray')
            plt.subplot(1, 2, 2), plt.imshow(img_label, cmap='gray')
            plt.show()

        y_true = tf.cast(img_label, tf.bool)
        y_pred = tf.cast(img_predicted, tf.bool)

        true_indices = tf.cast(tf.where(y_true), tf.float32)
        pred_indices = tf.cast(tf.where(y_pred), tf.float32)

        def compute_min_distance(set_a, set_b):
            """
            Compute the minimum distance from each point in set_a to the points in set_b.
            """
            expanded_set_a = tf.expand_dims(set_a, axis=1)
            expanded_set_b = tf.expand_dims(set_b, axis=0)
            distances = tf.reduce_min(tf.norm(expanded_set_a - expanded_set_b, axis=2), axis=1)
            return distances

        d_true_to_pred = compute_min_distance(true_indices, pred_indices)
        d_pred_to_true = compute_min_distance(pred_indices, true_indices)

        hausdorff_distance = tf.reduce_max(tf.stack([tf.reduce_max(d_true_to_pred), tf.reduce_max(d_pred_to_true)]))

        if hausdorff_distance.numpy() != np.inf:
            hausdorff_distance_sum += hausdorff_distance.numpy()
    print("Hausdorff Distance mean: ",hausdorff_distance_sum/ len(X_test_))
    hausdorff_distance_sum/ len(X_test_)
